## 参考資料
- [詳細確率ロボティクスのgit](https://github.com/ryuichiueda/LNPR_BOOK_CODES)


In [1]:
# conect to drive on colab
# from google.colab import drive
# drive.mount("/content/drive")
# dir_path="./drive/MyDrive/Colab Notebooks/ProbabilisticRobotics/"
dir_path="./"



In [3]:
import sys
sys.path.append(dir_path)
from ideal_robot import *
from scipy.stats import expon,norm
import numpy as np
pi=np.pi

### 不確かさの実装
- 雑音(ノイズ)
  - 指数分布(expon)の確率でドロー
  - ドローしたらposeにノイズを加える
- バイアス
  - ガウス分布(norm)から初期化時にドロー
  - ガウス分布の何%かをバイアスとして設定
  - 速度と角速度にバイアスを加える
- スタック
  - 指数分布(expon)の確率でドロー
  - スタックが起きるまでの時間の期待値、起きてから抜けるまでの時間の期待値をそれぞれ計算
  - スタック中は速度と角速度をゼロにする(現実のロボットはぶつかった衝撃で向きが変わったりするけど、ここでは簡略化)

In [39]:
class Robot(IdealRobot):
    def __init__(self,pose,agent=None,sensor=None,color="black",
    noise_per_meter=5,noise_std=pi/60,
    bias_rate_stds=(0.1,0.1),
    expected_escape_time=1e-100,expected_stuck_time=1e100):
        super().__init__(pose,agent,sensor,color)
        # noise
        self.noise_pdf=expon(scale=1.0/(1e-100+noise_per_meter))
        self.distance_until_noise=self.noise_pdf.rvs()
        self.theta_noise=norm(scale=noise_std)
        # bias
        self.bias_rate_nu=norm.rvs(loc=1.0,scale=bias_rate_stds[0])
        self.bias_rate_omega=norm.rvs(loc=1.0,scale=bias_rate_stds[1])        
        # stuck
        self.escape_pdf=expon(scale=expected_escape_time)
        self.stuck_pdf=expon(scale=expected_stuck_time)
        self.time_until_escape=self.noise_pdf.rvs()
        self.time_until_stuck=self.noise_pdf.rvs()
        self.is_stuck=False

    def noise(self,pose,nu,omega,time_interval):
        self.distance_until_noise-=abs(nu)*time_interval+self.r*abs(omega)*time_interval
        if self.distance_until_noise <= 0.0:
            self.distance_until_noise += self.noise_pdf.rvs()
            pose[2] += self.theta_noise.rvs()
        return pose

    def bias(self,nu,omega):
        return nu*self.bias_rate_nu,omega*self.bias_rate_omega

    def stuck(self,nu,omega,time_interval):
        if self.is_stuck:
            self.time_until_escape-=time_interval
            if self.time_until_escape <= 0.0:
                self.time_until_escape += self.escape_pdf.rvs()
            self.is_stuck=False
        else:
            self.time_until_stuck-=time_interval
            if self.time_until_stuck <= 0.0:
                self.time_until_stuck += self.stuck_pdf.rvs()
            self.is_stuck=True
        return nu*(not self.is_stuck),omega*(not self.is_stuck)

    def one_step(self,time_interval):
        if not self.agent: return
        obs = self.sensor.data(self.pose) if self.sensor else None
        nu,omega = self.agent.decision(obs)
        nu,omega=self.bias(nu,omega)
        nu,omega=self.stuck(nu,omega,time_interval)
        self.pose = self.state_transition(nu,omega,time_interval,self.pose)
        self.pose = self.noise(self.pose,nu,omega,time_interval)
    


In [45]:
world = World(10, 0.1)

circling = Agent(0.2, 10.0/180*pi)
no_bias_robot= IdealRobot( np.array([0, 0, 0]).T, sensor=None, agent=circling, color="red" )
world.append(no_bias_robot)

biased_robot= Robot( np.array([0, 0, 0]).T, sensor=None, agent=circling, color="gray" ,
noise_per_meter=0,
bias_rate_stds=(0.0,0.0),
expected_escape_time=60,
expected_stuck_time=60
)
world.append(biased_robot)

world.draw()